# Atmospheric Correction with SNAP

This notebook demonstrates how to perform atmospheric correction on satellite imagery using the SNAP (Sentinel Application Platform) Python API (`snappy`) in Python. Atmospheric correction removes atmospheric effects (e.g., scattering, absorption) to improve the accuracy of surface reflectance data.

## Prerequisites
- Install required libraries: `snappy`, `rasterio`, `numpy`, `matplotlib` (listed in `requirements.txt`).
- A Sentinel-2 Level-1C product (e.g., `S2_product.SAFE`). Replace the file path with your own data.
- SNAP software installed and configured with `snappy` (see `docs/installation.md`).

## Learning Objectives
- Apply atmospheric correction using SNAP’s Sen2Cor processor.
- Load and visualize the corrected Level-2A data.
- Save the corrected raster for further analysis.

In [ ]:
# Import required libraries
import snappy
from snappy import ProductIO
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os

## Step 1: Load Sentinel-2 Level-1C Product

Load a Sentinel-2 Level-1C product (.SAFE format) for atmospheric correction.

In [ ]:
# Define the path to the Sentinel-2 Level-1C product
product_path = 'S2_product.SAFE'

# Load the product
product = ProductIO.readProduct(product_path)

# Print product information
print(f'Product Name: {product.getName()}')
print(f'Product Type: {product.getProductType()}')

## Step 2: Apply Atmospheric Correction with Sen2Cor

Use the Sen2Cor processor in SNAP to perform atmospheric correction, converting Level-1C to Level-2A (surface reflectance).

In [ ]:
# Configure Sen2Cor parameters
parameters = snappy.HashMap()
parameters.put('resolution', '10')  # Output resolution (10m, 20m, or 60m)
parameters.put('outputDir', 'corrected_output')  # Output directory

# Run Sen2Cor
corrected_product = snappy.GPF.createProduct('Sen2Cor', parameters, product)

# Save the corrected product
output_path = 'corrected_output/S2_L2A_product.SAFE'
ProductIO.writeProduct(corrected_product, output_path, 'SENTINEL2')

print(f'Corrected product saved to: {output_path}')

## Step 3: Load and Visualize Corrected Data

Load the corrected Level-2A product and visualize an RGB composite (bands B4, B3, B2).

In [ ]:
# Load a specific band from the corrected product (e.g., 10m resolution)
# Adjust the path to the corrected GeoTIFF (e.g., B4 - red)
corrected_band_path = os.path.join(output_path, 'GRANULE', '*', 'IMG_DATA', 'R10m', '*_B04_10m.tif')

# Open with rasterio
with rasterio.open(corrected_band_path) as src:
    red = src.read(1).astype(float)
    bounds = src.bounds

# Visualize corrected band
plt.figure(figsize=(8, 8))
plt.imshow(red, cmap='gray', vmin=np.percentile(red, 2), vmax=np.percentile(red, 98))
plt.colorbar(label='Surface Reflectance')
plt.title('Corrected Red Band (B04)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 4: Create RGB Composite

Load bands B4 (red), B3 (green), and B2 (blue) from the corrected product to create an RGB composite.

In [ ]:
# Define paths to corrected bands (10m resolution)
red_path = os.path.join(output_path, 'GRANULE', '*', 'IMG_DATA', 'R10m', '*_B04_10m.tif')
green_path = os.path.join(output_path, 'GRANULE', '*', 'IMG_DATA', 'R10m', '*_B03_10m.tif')
blue_path = os.path.join(output_path, 'GRANULE', '*', 'IMG_DATA', 'R10m', '*_B02_10m.tif')

# Load bands
with rasterio.open(red_path) as src_red, rasterio.open(green_path) as src_green, rasterio.open(blue_path) as src_blue:
    red = src_red.read(1).astype(float)
    green = src_green.read(1).astype(float)
    blue = src_blue.read(1).astype(float)

# Stack and normalize for visualization
rgb = np.stack([red, green, blue], axis=-1)
rgb = rgb / np.percentile(rgb, 98) if np.percentile(rgb, 98) > 0 else rgb  # Normalize to 98th percentile
rgb = np.clip(rgb, 0, 1)  # Ensure values are in valid range

# Visualize RGB composite
plt.figure(figsize=(8, 8))
plt.imshow(rgb)
plt.title('Corrected RGB Composite (B04, B03, B02)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Next Steps

- Replace `S2_product.SAFE` with your own Sentinel-2 Level-1C product.
- Adjust Sen2Cor parameters (e.g., resolution, aerosol type) for your data.
- Explore other SNAP processors for preprocessing tasks.
- Proceed to the next notebook (`09_cloud_masking.ipynb`) for cloud masking.

## Notes
- Ensure SNAP and `snappy` are properly installed and configured.
- Verify the input product is a valid Sentinel-2 Level-1C .SAFE directory.
- Corrected bands are typically stored in the `IMG_DATA/R10m` folder of the output.
- See `docs/installation.md` for troubleshooting SNAP and `snappy` setup.